In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt


import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')

import pickle
with open('/Users/zachkeskinen/Documents/spicy-snow/data/banner_lidar.pkl', 'rb') as f:
    ds = pickle.load(f)
spicy_sd = ds.isel(time = 81)['snow_depth']

lidar = rxa.open_rasterio('/Users/zachkeskinen/Desktop/SNEX20_QSI_SD_0.5M_USIDBS_20210315_20210315.tif')
xmin, ymin, xmax, ymax = lidar.rio.bounds()
import pyproj
proj = pyproj.Transformer.from_crs(6340, 4326, always_xy=True)
xmin, ymin = proj.transform(xmin, ymin)
xmax, ymax = proj.transform(xmax, ymax)

spicy_sd = spicy_sd.rio.clip_box(*[xmin, ymin, xmax, ymax])

spicy_sd = spicy_sd.rio.write_nodata(np.nan)
lidar = lidar.rio.write_nodata(np.nan)

lidar = lidar.rio.reproject_match(spicy_sd)
spicy_sd = spicy_sd.where(~lidar.isnull())
lidar = lidar.squeeze(dim = 'band')

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt


import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')
from spicy_snow.download.forest_cover import download_fcf

import pickle
with open('/Users/zachkeskinen/Documents/spicy-snow/data/lidar_tmp.pkl', 'rb') as f:
    ds = pickle.load(f)

ds = ds.isel(time = slice(0,3))
s1_dA = ds.s1
ds = s1_dA.to_dataset()

In [ ]:
download_fcf(ds, out_fp= '/Users/zachkeskinen/Documents/spicy-snow/data/fcf.tif')

In [ ]:
"""
Functions to download IMS snow-coverage images.

https://usicecenter.gov/Products/ImsHome
https://nsidc.org/data/user-resources/help-center/how-access-data-using-ftp-client-command-line-wget-or-python
"""

import sys
import os
from os.path import basename, exists, expanduser, join
import shutil
from datetime import datetime
import urllib.request
from urllib.error import URLError
from tqdm import tqdm
from pyproj import Transformer
import shapely
import pandas as pd
import xarray as xr
import rioxarray as rxa

import logging
log = logging.getLogger(__name__)

sys.path.append(expanduser('~/Documents/spicy-snow'))
from spicy_snow.utils.download import url_download, decompress

def get_ims_day_data(year: str, doy: str, tmp_dir: str) -> xr.DataArray:
    """
    Download and decompress one days worth of IMS data.

    Args:
    year: Year of the data you want.
    doy: Calendar day of year you want in 'DDD' format. Range is 001 - 366.
    """

    # make temporary directory
    cd = os.getcwd()
    os.makedirs(tmp_dir, exist_ok = True)
    os.chdir(tmp_dir)

    # if we haven't found a file that works add one more to day and try again
    local_fp = None
    while not local_fp:
        try:
            # download IMS data for this day and year
            local_fp, _ = urllib.request.urlretrieve(f'ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02156/netcdf/1km/{year}/ims{year}{doy}_1km_v1.3.nc.gz', f'ims{year}{doy}_1km_v1.3.nc.gz')
        except:
            doy = f'{int(doy) + 1:03}'
    # decompress .gz compression
    out_file = decompress(local_fp, local_fp.replace('.gz',''))
    # open as xarray dataArray
    ims = rxa.open_rasterio(out_file, decode_times = False)
    os.chdir(cd)
    return ims


def download_snow_cover(dataset: xr.Dataset, tmp_dir: str = './tmp', clean: bool = True) -> xr.Dataset:
    """
    Download IMS snow-cover images.

    Args:
    dataset: Full dataset to add IMS data to
    tmp_dir: filepath to save temporary downloads to [default: './tmp']
    clean: Remove temporary directory after download?

    Returns:
    None
    """
    # get list of days that we have Sentinel-1 data for
    days = [pd.to_datetime(d) for d in dataset.time.values]

    all_ims = []
    for day in tqdm(days, desc = 'Downloading IMS snow-cover'):
        # download IMS data for this day and open with xarray
        ims = get_ims_day_data(day.year, f'{day.dayofyear:03}', tmp_dir = tmp_dir) #revert to clean = True at somepoint
        # add timestamp info
        ims = ims.assign_coords(time = [day])
        # reproject and clip to match dataset
        ims = ims.rio.reproject_match(dataset['s1'])
        # add day to list of ims days
        all_ims.append(ims)
    # make dataArray of all IMS images
    full_ims = xr.concat(all_ims, dim = 'time')

    # add dataArray of IMS to the sentinel-1 dataset with timestamp information
    dataset = xr.merge([dataset, full_ims.rename('ims')])
    
    # remove data directory of downloaded IMS tifs
    if clean == True:
        shutil.rmtree(tmp_dir)
    
    return dataset

# End of file

In [ ]:
ds1 = download_snow_cover(ds)

In [ ]:
ds1['ims']

In [ ]:
import matplotlib.pyplot as plt

def plot_xarray_2dhist(x, y, ax):
    x = x.values.ravel()
    y = y.values.ravel()

    xtmp = x[(~np.isnan(x) & ~np.isnan(y)) & (y != 0)]
    y = y[(~np.isnan(x) & ~np.isnan(y)) & (y != 0)]

    ax.hist2d(x = xtmp, y = y, bins = 150)

In [ ]:
fig, ax = plt.subplots()
plot_xarray_2dhist(lidar, spicy_sd, ax)
plt.title('Comparison of Banner March 15th Spicy SD and Lidar SD')
plt.xlim(0,5)
plt.ylim(0,5)

In [ ]:
fig, axes = plt.subplots(2, figsize = (12,8))

lidar.plot(ax = axes[0], vmin = 0, vmax = 4)
spicy_sd.plot(ax = axes[1], vmin = 0, vmax = 4)

axes[0].set_title("Lidar Snow Depth")
axes[1].set_title("Spicy Snow Depth")

for ax in axes:
    ax.set_xlabel('')
    ax.set_ylabel('')

In [ ]:
fig, ax = plt.subplots()
plot_xarray_2dhist(lidar, spicy_sd, ax)
plt.title('Comparison of Banner March 15th Spicy SD and Lidar SD')
plt.xlim(0,5)
plt.ylim(0,5)